In [1]:
# Steps 1–3: Full Dashboard with Filters + Data Table + Geo Chart + Pie Chart

from jupyter_dash import JupyterDash
from dash import Dash, dcc, html, Input, Output, dash_table
import pandas as pd
import plotly.express as px
from animal_shelter import AnimalShelter

# Set up MongoDB connection
shelter = AnimalShelter("aacuser", "aacpassword")

# Rescue filter specs
rescue_filters = {
    "Water Rescue": {
        "breeds": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"],
        "sex": "Intact Female",
        "age_min": 26,
        "age_max": 156
    },
    "Mountain or Wilderness Rescue": {
        "breeds": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 26,
        "age_max": 156
    },
    "Disaster or Individual Tracking": {
        "breeds": ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"],
        "sex": "Intact Male",
        "age_min": 20,
        "age_max": 300
    }
}

# Init app
app = JupyterDash(__name__)
app.title = "Grazioso Salvare Dashboard"

# Layout
app.layout = html.Div([
    # Header with logo, title, and your name
html.Div([
    html.A(
        html.Img(
            src='assets/Grazioso Salvare Logo.png',
            style={'height': '100px'}
        ),
        href='https://www.snhu.edu',
        target='_blank'
    ),
    html.H1("Grazioso Salvare: Animal Dashboard"),
    html.P("Dashboard developed by Bradly Grigg", style={"fontWeight": "bold", "fontSize": "16px"})
], style={'textAlign': 'center', 'padding': '10px'}),

    html.Div([
        html.Label("Select Rescue Type:"),
        dcc.RadioItems(
            id='rescue-filter',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset (Show All)', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        )
    ], style={'padding': '20px', 'width': '25%', 'display': 'inline-block', 'verticalAlign': 'top'}),

    html.Div([
        dash_table.DataTable(
            id='datatable',
            columns=[{"name": i, "id": i} for i in [
                'name', 'animal_type', 'breed', 'age_upon_outcome', 
                'sex_upon_outcome', 'location_lat', 'location_long'
            ]],
            page_size=10,
            style_table={'overflowX': 'auto'},
            style_cell={
                'textAlign': 'left',
                'minWidth': '100px', 'width': '100px', 'maxWidth': '200px',
                'whiteSpace': 'normal'
            },
        ),

        dcc.Graph(id='geo-chart'),
        dcc.Graph(id='breed-pie')
    ], style={'display': 'inline-block', 'width': '70%', 'verticalAlign': 'top'})
])

# Callback to update all widgets
@app.callback(
    Output('datatable', 'data'),
    Output('geo-chart', 'figure'),
    Output('breed-pie', 'figure'),
    Input('rescue-filter', 'value')
)
def update_dashboard(filter_option):
    if filter_option == 'Reset':
        results = shelter.read({}, limit=100)
    else:
        spec = rescue_filters[filter_option]
        query = {
            "animal_type": "Dog",
            "breed": {"$in": spec["breeds"]},
            "sex_upon_outcome": spec["sex"],
            "age_upon_outcome_in_weeks": {
                "$gte": spec["age_min"],
                "$lte": spec["age_max"]
            }
        }
        results = shelter.read(query)
        
    df = pd.DataFrame.from_records(results)

    # Columns to show
    columns = ['name', 'animal_type', 'breed', 'age_upon_outcome', 
               'sex_upon_outcome', 'location_lat', 'location_long']
    df = df[[col for col in columns if col in df.columns]]

    # --- GEO CHART ---
    if 'location_lat' in df.columns and 'location_long' in df.columns:
        fig_map = px.scatter_mapbox(
            df,
            lat="location_lat",
            lon="location_long",
            hover_name="breed",
            hover_data=["name", "sex_upon_outcome"],
            zoom=9,
            height=400
        )
        fig_map.update_layout(mapbox_style="open-street-map")
        fig_map.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    else:
        fig_map = {}

    # --- PIE CHART ---
    if 'breed' in df.columns and not df.empty:
        fig_pie = px.pie(
            df,
            names="breed",
            title="Breed Distribution"
        )
    else:
        fig_pie = {}

    return df.to_dict('records'), fig_map, fig_pie

# Run app
app.run_server(mode='inline', debug=True)